## Working with Dates and Times in Python

Working with date and times is a vital skill, because many data include date/time information, including:

- Weather data with dates and/or times.
- Computer logs with the timestamp for each event.
- Sales data with date/time range included.